In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Bidirectional, AveragePooling1D, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, CuDNNGRU , GlobalAveragePooling1D, Concatenate, TimeDistributed
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt

## CNN_BIGRU_TFIDF_BIGRAM
Copyright @I Gde Bagus Janardana Abasan

In [2]:
# Menampilkan informasi perangkat GPU yang digunakan
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    device_details = tf.config.experimental.get_device_details(device)
    print(device_details)
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

try:
    print(f'Found {physical_devices[0]} GPU(s)')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
    print('Memory growth set to True, using Nvidia Tesla T4')
except:
    print('Failed to set memory growth to True')
    pass

{'device_name': 'NVIDIA GeForce GTX 1660 Ti with Max-Q Design', 'compute_capability': (7, 5)}
Found PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') GPU(s)
Memory growth set to True, using Nvidia Tesla T4


In [3]:
def process_data(test_size):
  dataset = pd.read_csv('../../../data/data_preprocessed/dataset/DatasetHateSpeech_Final_TA2023.csv', usecols=['preprocess_final', 'preprocess_token', 'label_final'])
  # make data['label_fase_1'] to 0 and 1 binary classifier karena tensor hanya bisa input 0 =< label =< 1
  dataset['label_final'] = dataset['label_final'].apply(lambda x: 1 if x == 'HS' else 0)
  dataset.dropna(inplace=True)
  
  # feature extration
  # Create a CountVectorizer object and fit it to the training data
  vectorizer = TfidfVectorizer(ngram_range=(2,2), max_features=15000)
  vectorizer_data = vectorizer.fit_transform(dataset['preprocess_final'])

  # to_pd
  x_final = pd.DataFrame(vectorizer_data.toarray(), columns=vectorizer.get_feature_names_out())
  y_final = dataset['label_final'].copy()

  # split
  X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=test_size, random_state=0)
  
  return X_train, X_test, y_train, y_test

In [4]:
def create_cnn_bigru_model(X_train, y_train, X_test, y_test):
  # reshape# reshape train and test data
  X_train = np.array(X_train).reshape(X_train.shape[0], 1, X_train.shape[1])
  X_test = np.array(X_test).reshape(X_test.shape[0], 1, X_test.shape[1])

  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
  model.add(Bidirectional(GRU(32, activation='relu', return_sequences=True)))
  model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
  model.add(AveragePooling1D(pool_size=3, strides=2, padding='same'))
  model.add(Flatten())
  model.add(Dense(units = 32, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  opt = Adam(learning_rate=0.00001)
  model.compile(optimizer=opt,loss='binary_crossentropy', metrics=['accuracy'])

  input_shape = X_train.shape
  model.build(input_shape)
  model.summary()
  print()
  
  with tf.device('/cpu:0'):
    x = tf.convert_to_tensor(X_train, np.float32)
    y = tf.convert_to_tensor(y_train, np.float32)

  model.fit(x, y, epochs=10, batch_size=64, validation_data = (X_test, y_test))
  loss = model.history.history['loss']
  val_loss = model.history.history['val_loss']
  score = model.evaluate(X_test, y_test, batch_size=64, verbose=0)

  print()
  print('Validation Accuracy:', score[1])
  print('Validation Loss:', score[0])
  print()

  return model, loss, val_loss

In [5]:
def call():
  X_train, X_test, y_train, y_test = process_data(test_size=0.1)
  model, loss, val_loss = create_cnn_bigru_model(X_train, y_train, X_test, y_test)
  # predict
  X_test = np.array(X_test).reshape(X_test.shape[0], 1, X_test.shape[1])
  y_pred = model.predict(X_test)

  # confusion matrix
  classreport = classification_report(y_test, y_pred.round(), digits=4)
  print()
  print(classreport)
  print()
  accscore = accuracy_score(y_test, y_pred.round())
  precscore = precision_score(y_test, y_pred.round(), zero_division=0)
  recscore = recall_score(y_test, y_pred.round())
  f1score = f1_score(y_test, y_pred.round(), average='weighted', zero_division=0)

  return [accscore, precscore, recscore, f1score, loss, val_loss]

In [6]:
result = []
for i in range (5):
  result.append([i+1] + call())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (44856, 1, 32)            1440032   
                                                                 
 max_pooling1d (MaxPooling1D  (44856, 1, 32)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (44856, 1, 64)           12672     
 l)                                                              
                                                                 
 max_pooling1d_1 (MaxPooling  (44856, 1, 64)           0         
 1D)                                                             
                                                                 
 average_pooling1d (AverageP  (44856, 1, 64)           0         
 ooling1D)                                              

C:\Users\bjana\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bjana\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bjana\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (44856, 1, 32)            1440032   
                                                                 
 max_pooling1d_2 (MaxPooling  (44856, 1, 32)           0         
 1D)                                                             
                                                                 
 bidirectional_1 (Bidirectio  (44856, 1, 64)           12672     
 nal)                                                            
                                                                 
 max_pooling1d_3 (MaxPooling  (44856, 1, 64)           0         
 1D)                                                             
                                                                 
 average_pooling1d_1 (Averag  (44856, 1, 64)           0         
 ePooling1D)                                          

KeyboardInterrupt: 

In [ ]:
def loss_func(df):
  for idx, val in enumerate(df['No']):
    plt.plot(df['Loss'].iloc[idx])
    plt.plot(df['Val Loss'].iloc[idx])
    plt.title(f'loss function CNN+Bi-GRU TF-IDF BIGRAM-{idx+1}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.savefig(f'hasil/grafik/lossfunctionGraphics_CNN_BIGRU_BIGRAM-{idx+1}.png')
    plt.show()
    plt.clf()

In [ ]:
df = pd.DataFrame(result, columns=['No', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Loss', 'Val Loss'])
df.to_csv('hasil/hasil_CNN_BIGRU_BIGRAM.csv', index=False)

#average
print('Average Accuracy :', df['Accuracy'].mean())
print('Average Precision :', df['Precision'].mean())
print('Average Recall :', df['Recall'].mean())
print('Average F1 Score :', df['F1 Score'].mean())
loss_func(df)